# Neural Network by Scratch

### Importing Numpy and Pandas 

In [1]:
import numpy as np
import pandas as pd

### Loading the Dataset as a Pandas Dataframe

In [3]:
data = pd.read_csv('train.csv')
data.head(5)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Train and Validation set split
We need to first convert the dataframe into a numpy array and transpose it

In [10]:
data = np.array(data)
m , n = data.shape
print(f'data.shape = {m,n}\n')
np.random.shuffle(data)

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data[1:1000]

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:m]

print(f'X_train.shape = {X_train.shape}')
print(f'Y_train.shape = {Y_train.shape}')

data.shape = (42000, 785)

X_train.shape = (784, 41000)
Y_train.shape = (41000,)


### Initializing Parameters

In [20]:
def init_params():

    W1 = np.random.rand(10,784) - 0.5
    b1 = np.random.rand(10,1) - 0.5
    
    W2 = np.random.rand(10,10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    
    return W1, b1, W2, b2

### ReLu Function

In [13]:
def ReLu(Z):
    return np.maximum(Z,0)

### Softmax Function

In [14]:
def softmax(Z):
    return np.exp(Z)/sum(np.exp(Z))

### Forward Propagation

In [19]:
def forward_prop(W1, b1, W2, b2, X):

    Z1 = W1.dot(X) + b1
    A1 = ReLu(Z1)
    
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

### Derivative of ReLu

In [16]:
def deriv_relu(Z):
    return Z>0

### Backward Propagation

In [30]:
def back_prop(Z1, A1, Z2, A2, W2, X, Y):

    m = Y.shape[0]

    dZ2 = A2 - Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * sum(dZ2)
    
    dZ1 = W2.T.dot(dZ2) * deriv_relu(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * sum(dZ1)
    
    return dW1, db1, dW2, db2

### Update Parameters

In [31]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):

    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    
    return W1, b1, W2, b2

### Prediction and Accuracy functions

In [34]:
def make_prediction(A2):
    return np.argmax(A2, 0)

def get_accuracy(Y_hat, Y):
    print(Y_hat, Y)
    return np.sum(Y_hat==Y)/Y.size

### Training Function

In [35]:
def train(X, Y, iterations, alpha):
    
    W1, b1, W2, b2 = init_params()
    
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if (i%10 == 0):
            print(f'Iteration: {i}')
            print(f'Accuracy: {get_accuracy(make_prediction(A2),Y)}')
            
    return W1, b1, W2, b2

In [37]:
train(X_train/255, Y_train, 100, 0.1)

Iteration: 0
[6 6 6 ... 5 6 6] [7 5 1 ... 4 8 5]
Accuracy: 0.04624390243902439


/var/folders/11/qj1v9m9n6159r0f8tzq4w1zc0000gn/T/ipykernel_24716/149416460.py:2: RuntimeWarning: overflow encountered in exp
  return np.exp(Z)/sum(np.exp(Z))
/var/folders/11/qj1v9m9n6159r0f8tzq4w1zc0000gn/T/ipykernel_24716/149416460.py:2: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(Z)/sum(np.exp(Z))


Iteration: 10
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 20
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 30
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 40
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 50
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 60
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 70
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 80
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732
Iteration: 90
[0 0 0 ... 0 0 0] [7 5 1 ... 4 8 5]
Accuracy: 0.09807317073170732


(array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 array([[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]),
 array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
        [n